# 1. Importing libraries

In [84]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 2. Load data

In [85]:
column_names = [
    "erythema", "scaling", "definite borders", "itching", "koebner phenomenon",
    "polygonal papules", "follicular papules", "oral mucosal involvement", "knee and elbow involvement",
    "scalp involvement", "family history", "melanin incontinence", "eosinophils in the infiltrate",
    "PNL infiltrate", "fibrosis of the papillary dermis", "exocytosis", "acanthosis",
    "hyperkeratosis", "parakeratosis", "clubbing of the rete ridges", "elongation of the rete ridges",
    "thinning of the suprapapillary epidermis", "spongiform pustule", "munro microabcess",
    "focal hypergranulosis", "disappearance of the granular layer", "vacuolisation and damage of basal layer",
    "spongiosis", "saw-tooth appearance of retes", "follicular horn plug", "perifollicular parakeratosis",
    "inflammatory monoluclear inflitrate", "band-like infiltrate", "Age", "class"
]

dermatology_data = pd.read_csv('dermatology/dermatology.data', header=None, names=column_names)

# 3. Data Preprocessing
## 3.1 Data replace and fill missing values

In [86]:
# Replacing the missing values with NaN
dermatology_data.replace("?", pd.NA, inplace=True)
dermatology_data['Age'] = pd.to_numeric(dermatology_data['Age'], errors='coerce')

# Filling the missing values in 'Age' with the median age
age_median = dermatology_data['Age'].median()
dermatology_data['Age'].fillna(age_median, inplace=True)

# Dropping the 'perifollicular parakeratosis' column
dermatology_data.drop('perifollicular parakeratosis', axis=1, inplace=True)

## 3.2 Splitting the data into train and test sets

In [87]:
# Splitting the data into train and test sets
train, test = train_test_split(dermatology_data, test_size=0.2, random_state=42)

In [95]:
def split_dataset(data, feature_index, value):
        """
        根据特征索引和特征值分割数据集。
        :param data: DataFrame, 数据集。
        :param feature_index: int, 用于分割的特征的索引。
        :param value: 特征的值。
        :return: DataFrame, 分割后的子数据集。
        """
        feature_name = data.columns[feature_index]
        print(feature_name)
        reduced_data = data[data[feature_name] == value].drop(columns=[feature_name])
        return reduced_data

reduced_data = split_dataset(train, 1, 2)
reduced_data

scaling


,erythema,definite borders,itching,koebner phenomenon,polygonal papules,follicular papules,oral mucosal involvement,knee and elbow involvement,scalp involvement,family history,...,focal hypergranulosis,disappearance of the granular layer,vacuolisation and damage of basal layer,spongiosis,saw-tooth appearance of retes,follicular horn plug,inflammatory monoluclear inflitrate,band-like infiltrate,Age,class
202,2,0,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,3,0,52.0,5
75,3,1,2,0,0,0,0,0,0,0,...,0,0,0,2,0,0,2,0,25.0,2
16,2,1,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,30.0,2
66,2,2,0,0,0,0,0,2,2,0,...,0,3,0,0,0,0,3,0,36.0,1
271,2,2,0,0,0,0,0,0,0,0,...,0,1,0,2,0,0,3,0,48.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2,2,1,0,0,2,0,0,0,0,...,0,0,0,0,0,0,2,0,46.0,5
365,3,2,0,0,0,0,0,3,3,0,...,0,2,0,0,0,0,3,0,35.0,1
188,3,0,2,0,0,0,0,0,0,1,...,0,0,0,3,0,0,3,0,55.0,2
270,2,1,0,1,0,0,0,0,0,0,...,0,1,0,3,0,0,2,0,62.0,4


# 4. Decision Tree Model
## 4.1 Tools Preparation

In [0]:
def calculate_entropy(data):
    """
    Calculate the entropy of the data using ID3 formula.
    :param data: The dataset.
    :return: The entropy of the data.
    """
    labels = data.iloc[:, -1]
    label_counts = labels.value_counts()
    entropy = -sum((count/len(labels)) * np.log2(count/len(labels)) for count in label_counts)
    return entropy

def split_dataset(data, feature_index, value):
    """
    根据特征索引和特征值分割数据集。
    :param data: DataFrame, 数据集。
    :param feature_index: int, 用于分割的特征的索引。
    :param value: 特征的值。
    :return: DataFrame, 分割后的子数据集。
    """
    feature_name = data.columns[feature_index]
    reduced_data = data[data[feature_name] == value].drop(columns=[feature_name])
    return reduced_data

## 4.2 ID3DecisionTree

In [89]:
class ID3DecisionTree:
    def __init__(self, max_depth, min_info_gain):
        """
        Initialize the decision tree model.
        :param max_depth: The maximum depth of the tree which used to avoid over fitting.
        :param min_info_gain: The minimum information gain which used to avoid over fitting.
        """
        self.max_depth = max_depth
        self.min_info_gain = min_info_gain
        self.tree = None

    def choose_best_feature_to_split(self, data):
        """
        选择最佳特征进行数据集分割。
        :param data: DataFrame, 数据集。
        :return: int, 最佳特征的索引。
        """
        num_features = data.shape[1] - 1
        base_entropy = calculate_entropy(data)
        best_info_gain = 0.0
        best_feature = -1

        for i in range(num_features):
            feature_name = data.columns[i]
            unique_vals = set(data[feature_name])
            new_entropy = 0.0
            for value in unique_vals:
                sub_dataset = split_dataset(data, i, value)
                prob = len(sub_dataset) / float(len(data))
                new_entropy += prob * calculate_entropy(sub_dataset)

            info_gain = base_entropy - new_entropy
            if info_gain > best_info_gain and info_gain > self.min_info_gain:
                best_info_gain = info_gain
                best_feature = i

        return best_feature

    def build_tree(self, data, depth=0):
        """
        递归构建决策树。
        :param data: DataFrame, 数据集。
        :param depth: int, 当前深度。
        :return: dict, 构建的决策树。
        """
        labels = data.iloc[:, -1]
        if len(labels.unique()) == 1:
            return labels.iloc[0]
        if data.shape[1] == 1 or depth == self.max_depth:
            return labels.mode()[0]

        best_feature = self.choose_best_feature_to_split(data)
        if best_feature == -1:
            return labels.mode()[0]

        feature_name = data.columns[best_feature]
        tree = {feature_name: {}}
        for value in set(data[feature_name]):
            sub_data = split_dataset(data, best_feature, value)
            tree[feature_name][value] = self.build_tree(sub_data, depth + 1)

        return tree

    def fit(self, data):
        """
        训练决策树模型。
        :param data: DataFrame, 训练数据集。
        """
        self.tree = self.build_tree(data)

    def predict_sample(self, tree, sample):
        """
        对单个样本进行预测。
        :param tree: dict, 决策树。
        :param sample: Series, 单个样本数据。
        :return: 预测结果。
        """
        if not isinstance(tree, dict):
            return tree
        feature_index = list(tree.keys())[0]
        feature_value = sample[feature_index]
        sub_tree = tree[feature_index].get(feature_value, np.nan)
        return self.predict_sample(sub_tree, sample)

    def predict(self, data):
        """
        对数据集进行预测。
        :param data: DataFrame, 待预测的数据集。
        :return: Series, 预测结果。
        """
        predictions = data.apply(lambda x: self.predict_sample(self.tree, x), axis=1)
        return predictions

# 5. Model Training

In [90]:
# Training the decision tree model
tree = ID3DecisionTree(max_depth=5, min_info_gain=0.01)
tree.fit(train)

In [91]:
tree.tree

{'elongation of the rete ridges': {0: {'Age': {7.0: 6,
    8.0: 6,
    9.0: 6,
    10.0: {'follicular papules': {0: 2, 2: 6, 3: 6}},
    12.0: {'itching': {0: 4, 1: 6}},
    13.0: 6,
    15.0: 2,
    16.0: {'itching': {1: 2, 2: 4, 3: 3}},
    17.0: 4,
    18.0: {'scaling': {1: 4, 2: 3}},
    19.0: 2,
    20.0: {'definite borders': {0: 2, 1: 2, 2: 4}},
    21.0: {'erythema': {1: 4, 2: 2}},
    22.0: {'polygonal papules': {0: {'erythema': {1: 4, 2: 4, 3: 2}},
      2: 3,
      3: 3}},
    23.0: {'scaling': {1: 3, 2: 2}},
    25.0: {'spongiosis': {0: 3,
      1: 4,
      2: 2,
      3: {'definite borders': {0: 4, 2: 2}}}},
    26.0: 3,
    27.0: {'parakeratosis': {0: 3, 1: {'itching': {0: 4, 2: 5}}, 2: 2}},
    28.0: 3,
    30.0: {'koebner phenomenon': {0: {'erythema': {0: 5, 2: 2, 3: 2}},
      1: 3,
      2: 4}},
    31.0: {'erythema': {1: 4, 3: 2}},
    32.0: {'itching': {0: 4, 2: 3, 3: 3}},
    33.0: 2,
    34.0: {'koebner phenomenon': {0: 2, 1: 4, 3: 3}},
    35.0: {'itching': {0: {'

# 6. Model Evaluation
## 6.1 Accuracy

In [92]:
# Evaluating the decision tree model
predictions = tree.predict(test)
accuracy = (predictions == test['class']).mean()
print(f"Accuracy: {accuracy}")

Accuracy: 0.6351351351351351


## 6.2 MSE

In [93]:
# Evaluating the decision tree model
predictions = tree.predict(test)
mse = ((predictions - test['class']) ** 2).mean()
print(f"MSE: {mse}")

MSE: 0.6557377049180327
